# Text classification with the torchtext library

### Access to the raw dataset iterator

AG_NEWS dataset iterators는 원시 데이터를 레이블과 텍스트의 튜플로 생성한다.

AG_NEWS 데이터셋에는 4 종류의 레이블이 달려 있다.

1 : World   
2 : Sports   
3 : Business   
4 : Sci/Tec  

각 split의 데이터 건수는 
train: 120000, test: 7600 이다.

In [1]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(root='data', split='train')

In [2]:
train_iter.num_lines

120000

In [3]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

### Prepare data processing pipelines


다음은 tokenizer 및 vocabulary를 사용한 일반적인 자연어 데이터 처리의 예입니다.
첫 번째 단계는 원시 학습 데이터 세트로 vocabulary를 구축하는 것이다. `torchtext.vocab.vocab` 클래스의 옵션을 설정하여 사용자 정의의 vocabulary를 생성할 수 있다.

`build_vocab_from_iterator `를 사용하여 vocabulary를 생성한다.

참고) [torchtext.vocab.build_vocab_from_iterator](https://pytorch.org/text/stable/vocab.html#build-vocab-from-iterator)

In [35]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

# unk는 단어 집합에 없는 단어를 표현할 때 사용
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"]) 

token 리스트의 값을 정수로 변환할 수 있다.

In [36]:
[vocab[token] for token in ['here', 'is', 'an', 'example']]

[475, 21, 30, 5297]

text_pipeline은 vocab에 정의된 조회 테이블을 기반으로 텍스트 문자열을 정수 목록으로 변환한다. label_pipeline은 레이블을 정수로 변환합니다.

In [37]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) - 1

In [38]:
text_pipeline('here is the an example')

[475, 21, 2, 30, 5297]

In [39]:
label_pipeline('10')

9

### Generate data batch and iterator

`torch.utils.data.DataLoader `을 사용하여 원시 데이터를 데이터셋으로 생성할 수 있다.

텍스트 원소의 길이가 다를 수 있으므로, 데이터 배치와 오프셋을 생성하기 위한 사용자 함수 collate_batch()를 사용한다. 이 함수는 `torch.utils.data.DataLoader `의 `collate_fn` 인자로 넘겨준다.

`collate_fn` 의 입력은 그 크기가 batch_size인 텐서들의 리스트이며, `collate_fn` 은 이들을 미니배치로 묶는 역할을 한다. 

함수 collate_batch()는 앞서 선언한 `text_pipeline`과 `label_pipeline`을 사용한다.

In [40]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

max_len = 50

def collate_batch(batch, max_len=max_len):
    label_list, text_list = [], []

    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        text_holder = torch.ones(max_len, dtype=torch.int32) # fixed size tensor of max_len
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        pos = min(max_len, len(processed_text))
        text_holder[-pos:] = processed_text[-pos:]
        text_list.append(text_holder.unsqueeze(dim=0))

    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device)    

train_iter = AG_NEWS(root='data', split='train')

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

### Define the model

모형은 nn.Embedding과 뉴스 분류를 위한 linear layer로 구성된다. 

In [41]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.LSTM = nn.LSTM(embed_dim, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, num_class)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, _ = self.LSTM(embedded)
        last_output = output[:,-1,:]
        return self.fc(last_output) 

### Initiate instance

임베딩 차원이 64 인 모델을 생성한다. vocaburary 크기는 vocaburary 인스턴스의 길이와 같다. 클래스 수는 레이블 수와 같다.

In [42]:
num_class = 4
vocab_size = len(vocab)
emsize = 64
max_len = 50
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

### Define functions to train the model and evaluate results.


In [43]:
def train(dataloader):
    model.train()
    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predited_label = model(text)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

### Split the dataset and run the model

원래 AG_NEWS에는 valid 데이터 세트가 없으므로 훈련 데이터 세트를 0.95 (train) 및 0.05 (valid)의 분할 비율로 train / valid 세트로 분할한다.

In [44]:
# Hyperparameters
EPOCHS = 1 # epoch
LR = 0.001 # learning rate
BATCH_SIZE = 64 # batch size for training

In [45]:
from torch.utils.data.dataset import random_split

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

total_accu = None

train_iter, test_iter = AG_NEWS(root='data')
train_dataset = list(train_iter)
test_dataset = list(test_iter)
num_train = int(len(train_dataset) * 0.95)

split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)



In [46]:
a,b = next(iter(train_dataloader))
print(a,'\n', b)

tensor([2, 3, 0, 3, 2, 2, 1, 1, 3, 1, 0, 0, 2, 0, 1, 3, 0, 0, 1, 1, 0, 1, 1, 3,
        0, 2, 1, 3, 1, 2, 0, 3, 3, 3, 0, 0, 2, 2, 0, 3, 0, 1, 1, 0, 2, 3, 0, 1,
        0, 3, 3, 0, 2, 3, 2, 3, 0, 2, 0, 2, 2, 2, 2, 2]) 
 tensor([[   97,   236,     3,  ...,   475,    57,     1],
        [  800,     4, 38659,  ...,   163,    41,  1532],
        [    1,     1,     1,  ...,    10,    57,     1],
        ...,
        [   34,  1390,   124,  ...,  1340,   119,     1],
        [  148,    29,    50,  ...,   583,   611,     1],
        [    1,  8385,   572,  ...,    18,   919,     1]], dtype=torch.int32)


# Train the Model

In [47]:
print('Training the model...')
for epoch in range(1, EPOCHS + 1):
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    
    print('Epoch {:3d} : '
          'valid accuracy {:8.3f} '.format(epoch,
                                           accu_val))
    

Training the model...
Epoch   1 : valid accuracy    0.866 


### Evaluate the model with test dataset

In [48]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.863


### Test on a random news

In [49]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text_holder = torch.ones(max_len, dtype=torch.int32) 
        processed_text = torch.tensor(text_pipeline(text))
        pos = min(max_len, len(processed_text))
        text_holder[-pos:] = processed_text[-pos:]
        output = model(text_holder.unsqueeze(dim=0))
        return output.argmax(1).item() + 1

ex_text_str = "A pioneer in machine learning has argued that the technology is best placed to \
augment human intelligence and bemoaned ‘confusion’ over the meaning of \
artificial intelligence (AI). Michael I. Jordan, a professor in the department \
of electrical engineering and computer science, and department of statistics, \
at the University of California, Berkeley, told the IEEE that while \
science-fiction discussions around AI were ‘fun’, they were also a ‘distraction.’"

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sci/Tec news
